In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import models
from keras import layers


In [11]:
data = pd.read_csv('musicdata.csv', encoding = "ISO-8859-1")
data.head()

,filename,genre,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,000_Fallin.mp3,Blues,0.398766,0.261029,2173.414601,2348.635635,4469.697972,0.105094,-22.129293,98.975113,...,-5.252300,-0.315339,0.900259,2.533700,-2.005247,3.495977,0.705987,8.110969,1.611718,6.587331
1,001_ShortieLikeMine(feat.ChrisBrown&JohntáAust...,Blues,0.465902,0.270957,2518.655067,2651.224752,5488.783477,0.087210,-61.824787,78.159744,...,-3.559808,3.558018,-1.129447,5.634596,0.654691,3.739876,-5.944917,3.873111,-7.997187,3.063435
2,002_MoMoneyMoProblems(feat.PuffDaddy&Mase)-200...,Blues,0.418903,0.233998,2957.467102,2714.521091,6183.320944,0.152284,-41.089684,60.566902,...,-5.527019,4.102381,-3.110310,3.106595,-2.284698,5.771779,-4.016855,0.327673,-3.260499,-0.416478
3,003_IveChangedfeatKeyshiaCole.mp3,Blues,0.374477,0.258581,2747.243087,2849.698383,6255.395508,0.109958,-48.658676,74.215439,...,-2.504839,4.470598,2.079904,2.487600,-0.709448,5.637890,-2.080955,8.515167,-1.449764,8.345079
4,004_Blame.mp3,Blues,0.494732,0.372014,2064.441644,2600.087807,4853.437319,0.057051,-86.601624,94.769096,...,-5.616561,5.455497,5.883954,0.525487,-1.659588,5.081458,0.274451,5.138657,-1.814821,6.974661


In [12]:

# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
data.head()

,genre,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,Blues,0.398766,0.261029,2173.414601,2348.635635,4469.697972,0.105094,-22.129293,98.975113,-10.539597,...,-5.252300,-0.315339,0.900259,2.533700,-2.005247,3.495977,0.705987,8.110969,1.611718,6.587331
1,Blues,0.465902,0.270957,2518.655067,2651.224752,5488.783477,0.087210,-61.824787,78.159744,9.321901,...,-3.559808,3.558018,-1.129447,5.634596,0.654691,3.739876,-5.944917,3.873111,-7.997187,3.063435
2,Blues,0.418903,0.233998,2957.467102,2714.521091,6183.320944,0.152284,-41.089684,60.566902,-7.332506,...,-5.527019,4.102381,-3.110310,3.106595,-2.284698,5.771779,-4.016855,0.327673,-3.260499,-0.416478
3,Blues,0.374477,0.258581,2747.243087,2849.698383,6255.395508,0.109958,-48.658676,74.215439,23.380754,...,-2.504839,4.470598,2.079904,2.487600,-0.709448,5.637890,-2.080955,8.515167,-1.449764,8.345079
4,Blues,0.494732,0.372014,2064.441644,2600.087807,4853.437319,0.057051,-86.601624,94.769096,28.982912,...,-5.616561,5.455497,5.883954,0.525487,-1.659588,5.081458,0.274451,5.138657,-1.814821,6.974661


In [20]:
genre_list = data.iloc[:, 0]
genre_list

encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 ... 5 5 5]


In [22]:

# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, 1:-1], dtype = float))

# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# creating a model
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                    
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

# predictions
predictions = model.predict(X_test)
np.argmax(predictions[0])


Epoch 1/20
16/16 [==============================] - 2s 8ms/step - loss: 1.8881 - accuracy: 0.3800
Epoch 2/20
16/16 [==============================] - 0s 10ms/step - loss: 1.4738 - accuracy: 0.4795
Epoch 3/20
16/16 [==============================] - 0s 12ms/step - loss: 1.2794 - accuracy: 0.5378
Epoch 4/20
16/16 [==============================] - 0s 10ms/step - loss: 1.1593 - accuracy: 0.5698
Epoch 5/20
16/16 [==============================] - 0s 10ms/step - loss: 1.0778 - accuracy: 0.6038
Epoch 6/20
16/16 [==============================] - 0s 9ms/step - loss: 1.0293 - accuracy: 0.6175
Epoch 7/20
16/16 [==============================] - 0s 9ms/step - loss: 0.9701 - accuracy: 0.6449
Epoch 8/20
16/16 [==============================] - 0s 6ms/step - loss: 0.9242 - accuracy: 0.6535
Epoch 9/20
16/16 [==============================] - 0s 7ms/step - loss: 0.9080 - accuracy: 0.6560
Epoch 10/20
16/16 [==============================] - 0s 6ms/step - loss: 0.8729 - accuracy: 0.6707
Epoch 11/20
16/

7